In [1]:
import numpy as np
import tellurium as te
import controlSBML as ctl
import pandas as pd

import controlSBML as ctl

In [2]:
MDL = '''
J0: S0 -> S1; S0
J1: S1 -> S2; S1
J2: S2 -> S3; S2
J3: S2 -> S3; S3
J0a: S0a -> S1a; S0a
J1a: S1a -> S2a; S1a
J2a: S2a -> S3a; S2a
J3a: S2a -> S3a; S3a
S0a = 10
S1a = 0
S2a = 0
S3a = 0
S0 = 10
S1 = 0
S2 = 0
S3 = 0
'''
RR = te.loada(MDL)
data = RR.simulate(0, 1)

In [3]:
ctlsb = ctl.ControlSBML(MDL)

In [4]:
state_space = ctlsb.makeStateSpace()
ctl.mat2DF(state_space.A, column_names=ctlsb.state_names, row_names=ctlsb.state_names)

,S1,S2,S3,S1a,S2a,S3a
S1,-1.0,0.0,0.0,1.0,-1.0,0.0
S2,0.0,-1.0,1.0,0.0,0.0,-1.0
S3,0.0,-1.0,-2.0,0.0,0.0,-1.0
S1a,-1.0,0.0,0.0,-2.0,-1.0,0.0
S2a,1.0,0.0,0.0,0.0,1.0,0.0
S3a,0.0,1.0,0.0,0.0,0.0,1.0


In [5]:
ctlsb.output_ser

S0     10.0
S1      0.0
S2      0.0
S3      0.0
S0a    10.0
S1a     0.0
S2a     0.0
S3a     0.0
dtype: float64

In [6]:
RR.conservedMoietyAnalysis = True
ctl.ppMat (RR.getFullJacobian())
print("\n")
ctl.ppMat (RR.getReducedJacobian())

      S2  S2a           S1a            S1   S3  S3a   S0  S0a
S2  -1.0  0.0  0.000000e+00  1.000000e+00 -1.0  0.0  0.0  0.0
S2a  0.0 -1.0  1.000000e+00  0.000000e+00  0.0 -1.0  0.0  0.0
S1a  0.0  0.0 -1.000000e+00  0.000000e+00  0.0  0.0  0.0  1.0
S1   0.0  0.0  0.000000e+00 -1.000000e+00  0.0  0.0  1.0  0.0
S3   1.0  0.0  0.000000e+00  0.000000e+00  1.0  0.0  0.0  0.0
S3a  0.0  1.0  0.000000e+00  0.000000e+00  0.0  1.0  0.0  0.0
S0   0.0  0.0  0.000000e+00 -3.219093e-15  0.0  0.0 -1.0  0.0
S0a  0.0  0.0 -3.219093e-15  0.000000e+00  0.0  0.0  0.0 -1.0


      S2  S2a  S1a   S1   S3  S3a
S2  -1.0  0.0  0.0  1.0 -1.0  0.0
S2a  0.0 -1.0  1.0  0.0  0.0 -1.0
S1a  0.0 -1.0 -2.0  0.0  0.0 -1.0
S1  -1.0  0.0  0.0 -2.0 -1.0  0.0
S3   1.0  0.0  0.0  0.0  1.0  0.0
S3a  0.0  1.0  0.0  0.0  0.0  1.0


In [7]:
L0 = RR.getL0Matrix()
print(L0)

       S2, S2a, S1a, S1, S3, S3a
S0  [[ -1,   0,   0, -1, -1,   0],
S0a  [  0,  -1,  -1,  0,  0,  -1]]



In [8]:
RR.getConservedMoietyValues()

array([10., 10.])

In [9]:
RR.getConservedMoietyIds()

('_CSUM0', '_CSUM1')

In [10]:
values = [RR.getValue(s) for s in RR.getIndependentFloatingSpeciesIds()]
arr = np.array(values)
dependent_arr = np.matmul(L0, arr)
dependent_arr

[-6.32120543 -6.32120543]

In [11]:
np.matmul (L0, S_I) + RR.getConservedMoietyValues()

NameError: name 'S_I' is not defined

In [ ]:
# Construct C matrix
jacobian_mat = RR.getReducedJacobian()
names = jacobian_mat.rownames
jacobian_df = pd.DataFrame(jacobian_mat, columns=names, index=names)
C_df = pd.DataFrame(np.eye(len(names)), columns=names, index=names)
L0 = RR.getL0Matrix()
L0_df = pd.DataFrame(L0, columns=L0.colnames, index=L0.rownames)
C_df = pd.concat([C_df, L0_df], axis=0)
C_df